In [ ]:
# %matplotlib inline
# import necessary libraries 
import os

import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import time
import torchaudio
import psutil

matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


print(torch.__version__)
print(torchaudio.__version__)
print(device)

# Downloading a sound file and saving it in proper directory
SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"  # noqa: E501
SPEECH_FILE = "/content/speech.wav"

if not os.path.exists(SPEECH_FILE):
   os.makedirs("_assets", exist_ok=True)
   with open(SPEECH_FILE, "wb") as file:
          file.write(requests.get(SPEECH_URL).content)


1.13.1+cu116
0.13.1+cu116
cpu


In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H  

#print("Sample Rate:", bundle.sample_rate)

#print("Labels:", bundle.get_labels())

In [ ]:
model = bundle.get_model().to(device)

print(model.__class__)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [ ]:
#IPython.display.Audio(SPEECH_FILE)

In [ ]:
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)


In [ ]:
#print(waveform)

In [ ]:
#waveform.shape

In [ ]:
#print(sample_rate)

In [ ]:
with torch.inference_mode():
    features, _ = model.extract_features(waveform)

In [ ]:
#features

In [ ]:
#len(features)

In [ ]:
#fig, ax = plt.subplots(len(features), 1, figsize=(16, 4.3 * len(features)))
#for i, feats in enumerate(features):
    #ax[i].imshow(feats[0].cpu())
    #ax[i].set_title(f"Feature from transformer layer {i+1}")
    #ax[i].set_xlabel("Feature dimension")
    #ax[i].set_ylabel("Frame (time-axis)")
#plt.tight_layout()
#plt.show()

In [ ]:
with torch.inference_mode():
    emission, _ = model(waveform)

In [ ]:
#print(emission.shape)

In [ ]:
#len(bundle.get_labels())

In [ ]:
#plt.imshow(emission[0].cpu().T)
#plt.title("Classification result")
#plt.xlabel("Frame (time-axis)")
#plt.ylabel("Class")
#plt.show()
#print("Class labels:", bundle.get_labels())

In [ ]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [ ]:
#indices = torch.argmax(emission, dim=-1)
#indices = torch.unique_consecutive(indices, dim=-1)

#print(indices.shape)

In [ ]:
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
#transcript = decoder(emission[0])

In [ ]:
#print(transcript)
#IPython.display.Audio(SPEECH_FILE)

In [ ]:
model_path = '/content/asr.onnx'

In [ ]:
torch.save(model,model_path)

In [ ]:
#print model size using model_size function
def model_size(file_name):
    file_stats = os.stat(file_name)

    #print(file_stats)
    print("SIZE OF MODEL")
    print(f'File Size in Bytes is {file_stats.st_size}')
    print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

In [ ]:
model_size("/content/asr.onnx")

SIZE OF MODEL
File Size in Bytes is 396533691
File Size in MegaBytes is 378.16399669647217


In [ ]:
cpu_info = !lscpu
for inf_item in cpu_info.get_list():
  print(inf_item)

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [ ]:
# To get to memory and cpu information
import psutil
split_bar = '='*20
memory_info = psutil.virtual_memory()._asdict()
print(f"{split_bar} Memory Usage {split_bar}")
for k,v in memory_info.items():
  print(k, v)
print(f"{split_bar} CPU Usage {split_bar}")
print(f"CPU percent: {psutil.cpu_percent()}%")

==================== Memory Usage ====================
total 13616332800
available 9472606208
percent 30.4
used 3814289408
free 6314266624
active 1353936896
inactive 5630181376
buffers 422297600
cached 3065479168
shared 1347584
slab 237436928
==================== CPU Usage ====================
CPU percent: 40.2%


In [ ]:
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import onnx
import onnxruntime

In [ ]:
input_size = 100000

In [ ]:
dummy_input = torch.randn(1, input_size, requires_grad=True)

In [ ]:
torch.onnx.export(model,         # model being run
         dummy_input,       # model input (or a tuple for multiple inputs)
         "asr3.onnx",       # where to save the model
         export_params=True,  # store the trained parameter weights inside the model file
         opset_version=10,    # the ONNX version to export the model to
         do_constant_folding=True,  # whether to execute constant folding for optimization
         input_names = ['modelInput'],   # the model's input names
         output_names = ['modelOutput'], # the model's output names
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes
                                'modelOutput' : {0 : 'batch_size'}})

In [ ]:
cpu_info = !lscpu
for inf_item in cpu_info.get_list():
  print(inf_item)

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [ ]:
# To get to memory and cpu information
import psutil
split_bar = '='*20
memory_info = psutil.virtual_memory()._asdict()
print(f"{split_bar} Memory Usage {split_bar}")
for k,v in memory_info.items():
  print(k, v)
print(f"{split_bar} CPU Usage {split_bar}")
print(f"CPU percent: {psutil.cpu_percent()}%")

==================== Memory Usage ====================
total 13616332800
available 9181470720
percent 32.6
used 4105424896
free 5414563840
active 1354211328
inactive 6517055488
buffers 422522880
cached 3673821184
shared 1347584
slab 255778816
==================== CPU Usage ====================
CPU percent: 65.5%


In [ ]:
#decoder = GreedyCTCDecoder(labels=bundle.get_labels())
#transcript = decoder(emission[0])

In [ ]:
#print(transcript)

In [ ]:
#emission[0].shape

In [ ]:
#print(transcript)
#IPython.display.Audio(SPEECH_FILE)

In [ ]:
ort_session = onnxruntime.InferenceSession('asr3.onnx') # load onnx model

In [ ]:
import soundfile as sf
from scipy.io import wavfile
import scipy.signal as sps
import os


In [ ]:
!pip install transformers onnxruntime onnx pythainlp soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pythainlp.util import normalize

In [ ]:
def _normalize(x): #
  """You must call this before padding.
  Code from https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/processor.py#L101
  Fork TF to numpy
  """
  # -> (1, seqlen)
  mean = np.mean(x, axis=-1, keepdims=True)
  var = np.var(x, axis=-1, keepdims=True)
  return np.squeeze((x - mean) / np.sqrt(var + 1e-5))

In [ ]:
#sampling_rate, data = wavfile.read(SPEECH_FILE)
#samples = round(len(data) * float(new_rate) / sampling_rate)
#new_data = sps.resample(data, samples)
#speech = np.array(new_data, dtype=np.float32)
#speech = _normalize(speech)[None]
#padding = np.zeros((speech.shape[0], AUDIO_MAXLEN - speech.shape[1]))
#speech = np.concatenate([speech, padding], axis=-1).astype(np.float32)
#ort_inputs = {"modelInput": speech}
#ort_outs = ort_session.run(None, ort_inputs)
#prediction = np.argmax(ort_outs, axis=-1)

In [ ]:
#ort_outs = torch.tensor(ort_outs[0])

In [ ]:
#ort_outs

In [ ]:
#ort_outs[0].shape

In [ ]:
#transcript = decoder(ort_outs[0])

In [ ]:
#len(transcript)

In [ ]:
#print(transcript)

# **FUNCTION FOR SPEECH TO TEXT FROM SPEECH FILE**

In [ ]:
def asr(Speech_file):
    new_rate = 16000
    AUDIO_MAXLEN = input_size
    start_time=time.time()
    sampling_rate, data = wavfile.read(SPEECH_FILE)
    samples = round(len(data) * float(new_rate) / sampling_rate)
    new_data = sps.resample(data, samples)
    speech = np.array(new_data, dtype=np.float32)
    speech = _normalize(speech)[None]
    padding = np.zeros((speech.shape[0], AUDIO_MAXLEN -speech.shape[1]))
    speech = np.concatenate([speech, padding], axis=-1).astype(np.float32)
    ort_inputs = {"modelInput": speech}
    ort_outs = ort_session.run(None, ort_inputs)
    ort_outs = torch.tensor(ort_outs[0])
    transcript = decoder(ort_outs[0])
    
    inference_time = time.time() - start_time

    # Measure RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2) # in MB   
    
    c=" "
    transcript_new=""
    for i in transcript:
      if i =="|":
        transcript_new+=c
      else:
        transcript_new+=i
    print("inference_time", inference_time )
    print('ram_usage',ram_usage)
    print(transcript_new)
    



In [ ]:
import torch
import time

# Set the model to evaluation mode
model.eval()

# Create a dummy input tensor
input_tensor = torch.randn(1, 100000)

# Measure the time it takes to process the input tensor
start_time = time.time()
with torch.no_grad():
    _ = model(input_tensor)
latency = time.time() - start_time

print(f"Latency: {latency:.5f} seconds")


Latency: 3.23490 seconds


In [ ]:
asr(SPEECH_FILE)

inference_time 1.7718555927276611
ram_usage 3464.5703125
I HAD THAT CURIOSITY BESIDE ME AT THIS MOMENT 


In [ ]:
!pip install scalene

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.2/741.2 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1


In [ ]:
import scalene

In [ ]:
!pip install scalene

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import scalene

In [ ]:


!ls

asr3.onnx  asr.onnx  _assets  model.py	sample_data  speech.wav  untitled27.py


In [ ]:

%load_ext scalene

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version.


In [ ]:
! scalene /content/untitled27.py --html >> model-scalene1.html

In [ ]:
from IPython.display import display, HTML, IFrame
display(HTML("/content/model-scalene.html"))

In [ ]:
! scalene /content/model (1).py --html >> model-scalene.html

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: ` scalene /content/model (1).py --html >> model-scalene.html'


In [ ]:
from IPython.display import display, HTML, IFrame
display(HTML("/content/model-scalene.html"))

CPU PERCENTAGE

In [ ]:
import psutil
import torch
import torchaudio

# Load audio file and convert to tensor
audio_file = "/content/speech.wav"
waveform, sample_rate = torchaudio.load(audio_file)
features, _ = model.extract_features(waveform)
with torch.inference_mode():
    emission, _ = model(waveform)
indices = torch.argmax(emission, dim=-1)  # [num_seq,]
indices = torch.unique_consecutive(indices, dim=-1)
#indices = [i for i in indices if i != 0]
audio_tensor = indices

# Load model
#model = torch.hub.load('pytorch/fairseq', 'wav2vec2_base', pretrained=True)

# Start CPU monitoring
process = psutil.Process()
cpu_percent_list = []

for i in range(100):
    # Run inference and measure CPU usage
    with torch.no_grad():
        output = model.feature_extractor(waveform).transpose(1, 2)
        
        output = model.feature_aggregator(output)
    cpu_percent_list.append(process.cpu_percent())

# Print CPU usage stats
print("Average CPU usage: {}%".format(sum(cpu_percent_list) / len(cpu_percent_list)))


TypeError: ignored

**print the onnx model size**

In [ ]:
model_size("/content/asr3.onnx") 

SIZE OF MODEL
File Size in Bytes is 377833426
File Size in MegaBytes is 360.33003425598145


**WORKING WIH MICROPHONES**

In [ ]:
import sys

print(sys.version)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


In [ ]:
!sudo apt-get install python3-pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pyaudio is already the newest version (0.2.11-1.1build1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
!sudo apt-get install python-gnuradio-audio-portaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-gnuradio-audio-portaudio


In [ ]:
import pyaudio

pa = pyaudio.PyAudio()

print('\navailable devices:')

for i in range(pa.get_device_count()):
    dev = pa.get_device_info_by_index(i)
    name = dev['name'].encode('utf-8')
    print(i, name, dev['maxInputChannels'], dev['maxOutputChannels'])

print('\ndefault input & output device:')
print(pa.get_default_input_device_info())
print(pa.get_default_output_device_info())


available devices:

default input & output device:


OSError: ignored

In [ ]:
import pyaudio
import wave

# Set parameters for recording
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

# Initialize PyAudio object
audio = pyaudio.PyAudio()

# Open microphone stream
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

print("Recording started...")

# Create a buffer to store the recorded data
frames = []

# Record audio data for the specified time
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording finished.")

# Stop and close the stream
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded data to a WAV file
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print("Audio file saved as " + WAVE_OUTPUT_FILENAME)


OSError: ignored

In [ ]:

# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
record()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\xe5vG0\x9c(L\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3A\xdf\x81\x00\x00\x80\xfb\x83\xdc\xa2\x7f\xfd\x04\xf5}%\xc9r\xf0\xf5\xae\x82\x89*s\xfa\xb8\x9c\xa3\xdb;\x9f\xc9\xa47\xfa\x92\xda\x83\xb6\xfd\x035\xd5a\xcfZ\x1a\xac\xf7Q\xbd\x8ao\x04\r\xaaN)\x98\x15\xf1p\xf4\xb2\xc7\xa6Vl\x95a\xf5\x84\x02\x11Z\xa4\x11\x90\x87}I>\xb9\xd2\xef}A[\x9fh\xe8!7\xf1\xd4\xe9,\xbc6\xc3\n\xaa\xf1T\xf5D\xfej\xbf\xa0y\xd9\x94\xc6\xc1\xec\xee\xcbx\x02l!_l\x89\xf2\xa6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\xc2!l\x8fD@\x80\xf9\xc6\x82\xe0\xa1\xab\xd3\xdfd2a?\xf3\x96\x16V\xd6N\x98\x1e

In [ ]:
%pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00


In [ ]:

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()